<a href="https://colab.research.google.com/github/dConall/Monopoly/blob/master/Homework_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 2

### To do list
* Preprocessing
    * Tokenisation (Are we able to keep words such as wi-fi as one token instead of two tokens wi and fi)
    * Remove stopwords, punctuation, numbers (Or covert to textual representations)
    * Stemming
    * Lemmatization
    * get noun tokens
    * bi-grams/tri-grams
    * Label Encoding
    * Split training into train and validation set
    * Could Someone convert the preprocessing into one function. That way we can use it again when preprocessing the test data
* Feature Engineering
    * TD-IDF
    * Word Embeddings
    * Count vectors
    * Bag of Words
* Classification
    * Naive bayes
    * SVM
    * Logistic regression
    * random forest

In [28]:
##importing libraries
import numpy as np
import nltk, io, os
from nltk import ngrams
import nltk
nltk.download('stopwords'), nltk.download('averaged_perceptron_tagger'), nltk.download('wordnet')
import pandas as pd
import warnings
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
##importing dataset
raw_trainset = pd.read_csv('trainingset.csv',sep='^',header=0)

In [30]:
raw_trainset.head()

,content,category
0,French boss to leave EADS The French co-head o...,business
1,"Gamers could drive high-definition TV, films, ...",tech
2,Stalemate in pension strike talks Talks aimed ...,politics
3,Johnny and Denise lose Passport Johnny Vaughan...,entertainment
4,Tautou 'to star in Da Vinci film' French actre...,entertainment




## Preprocessing


Removing  punctuation, numbers, and stopwords from the texts.

In [0]:
##adding new column to dataset for tokens
temp = pd.DataFrame([],columns=['tokens'])
raw_trainset = raw_trainset.join(temp)

In [32]:
##importing tokenizer and stopword dictionary
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

stopwords = set(stopwords.words('english'))

for index, row in raw_trainset.iterrows():
    
    ##getting token words from the articles while removing punctuation
    pattern = r'\w+'
    tokenizer = RegexpTokenizer(pattern)
    token_words = tokenizer.tokenize(row['content'].replace("-", "_")) # -> Replacing hyphens with underscores so that words like
                                                                       #    Co-Head, Wi-Fi, High-Definition etc. stay together
    
    ##removong numbers
    token_words = ([word for word in token_words if not word.isdigit()])
    
    ##removing stopwords
    token_words = [word for word in token_words if not word in stopwords] 
    
    ##Stemming
    porter = PorterStemmer()
    token_words = [porter.stem(word) for word in token_words]
    
    ##Transfer tag function
    def transfer_tag(treebank_tag):
        if treebank_tag.startswith('j' or 'J'):
            return 'a'
        elif treebank_tag.startswith('v' or 'V'):
            return 'v'
        elif treebank_tag.startswith('n' or 'N'):
            return 'n'
        elif treebank_tag.startswith('r' or 'R'):
            return 'r'
        else:
            # As default pos in lemmatization is Noun
            return 'n'
        
    ##Lemmatization
    wnl = WordNetLemmatizer()

    lemma_words = []
    for word, tag in nltk.pos_tag(token_words):
        firstletter = tag[0].lower() # -> get the first letter of tag and put them decapitalized form
        wtag = transfer_tag(firstletter) # -> extract the word's tag (noun, verb, adverb, adjective)
        if not wtag:
            lemma_words.extend([word])
        else:
            lemma_words.extend([wnl.lemmatize(word, wtag)])
    
    raw_trainset.set_value(index, 'tokens', lemma_words)

raw_trainset.head()

,content,category,tokens
0,French boss to leave EADS The French co-head o...,business,"[french, bos, leav, ead, the, french, co_head,..."
1,"Gamers could drive high-definition TV, films, ...",tech,"[gamer, could, drive, high_definit, TV, film, ..."
2,Stalemate in pension strike talks Talks aimed ...,politics,"[stalem, pension, strike, talk, talk, aim, ave..."
3,Johnny and Denise lose Passport Johnny Vaughan...,entertainment,"[johnni, denis, lose, passport, johnni, vaugha..."
4,Tautou 'to star in Da Vinci film' French actre...,entertainment,"[tautou, star, Da, vinci, film, french, actres..."


In [0]:
#One hot encoding the label 
trainset = pd.get_dummies(raw_trainset, columns=["category"]).head()

In [9]:
trainset

,content,tokens,category_business,category_entertainment,category_politics,category_sport,category_tech
0,French boss to leave EADS The French co-head o...,"[French, boss, leave, EADS, The, French, co, h...",1,0,0,0,0
1,"Gamers could drive high-definition TV, films, ...","[Gamers, could, drive, high, definition, TV, f...",0,0,0,0,1
2,Stalemate in pension strike talks Talks aimed ...,"[Stalemate, pension, strike, talks, Talks, aim...",0,0,1,0,0
3,Johnny and Denise lose Passport Johnny Vaughan...,"[Johnny, Denise, lose, Passport, Johnny, Vaugh...",0,1,0,0,0
4,Tautou 'to star in Da Vinci film' French actre...,"[Tautou, star, Da, Vinci, film, French, actres...",0,1,0,0,0
